In [ ]:
#Task 1 Solution
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [ ]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(-90.000,90.000,size=1500)
lngs = np.random.uniform(-180.000, 180.000,size=1500)

lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
coordinates = list(lat_lngs)

In [ ]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
        city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
        # If the city is unique, then we will add it to the cities list.
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

In [ ]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Import the datetime module from the datetime library.
from datetime import datetime

In [ ]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    if(i%50 == 0 and i >= 50):
        set_count+=1
        record_count = 1
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count+=1


# Run an API request for each of the cities.
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["main"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        #Get rain or snow details if present
        try:
            if city_weather_description == 'Rain':
                city_rain = city_weather['rain']['3h']
            else:
                city_rain = 0
        except:
            print("No rain attribute found")
        
        try:
            if city_weather_description == 'Snow':
                city_snow = city_weather['snow']['3h']
            else:
                city_snow = 0
        except:
            print("No snow attribute found")
            
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_weather_description,
                          "Rain inches(last 3hrs)":city_rain,
                          "Snow inches(last 3hrs)":city_snow,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
columns = ["City","Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description",
          "Rain inches(last 3hrs)", "Snow inches(last 3hrs)"]
city_data_df = city_data_df[columns]
city_data_df.head(10)

In [ ]:
# Create the output file (CSV).
output_data_file = "data/WeatherPy_challenge.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
rain_cities = city_data_df.loc[(city_data_df["Rain inches(last 3hrs)"] > 0)]
snow_cities = city_data_df.loc[(city_data_df["Snow inches(last 3hrs)"] > 0)]
rain_or_snow_cities = city_data_df.loc[(city_data_df["Rain inches(last 3hrs)"] > 0) 
                                       |(city_data_df["Snow inches(last 3hrs)"] > 0)]
rain_or_snow_cities.head()

In [ ]:
rain_cities.head()

In [ ]:
snow_cities.head()

In [ ]:
# Rainfall or Snow recorded cities count
rain_or_snow_cities.count()

In [ ]:
#Rainfall recorded cities count
rain_cities.count()

In [ ]:
#Snow recorded cities count
snow_cities.count()